In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Problem Statement:
You work in XYZ Company as a Python Data Scientist. The company officials
have collected some data on health parameters based on diabetes and wish for
you to create a model from it.
# Dataset:
diabetes.csv
# Tasks To Be Performed:
1. Load the dataset using pandas
2. Extract data from outcome column is a variable named Y
3. Extract data from every column except outcome column in a variable
named X
4. Divide the dataset into two parts for training and testing in 70% and 30%
proportion
5. Create and train Logistic Regression Model on training set
6. Make predictions based on the testing set using the trained model
7. Check the performance by calculating the confusion matrix and accuracy
score of the model

In [32]:
# 1
df = pd.read_csv("/content/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [34]:
df.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [35]:
# 2
y = df['Outcome']
# 3
X = df.drop(columns='Outcome',axis=1)

In [36]:
# 4
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size = 0.3, random_state = 2)

In [37]:
# 5
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

In [38]:
# 6
y_pred = lr.predict(x_test)

In [39]:
# 7
from sklearn.metrics import accuracy_score,confusion_matrix

print(f"Accuracy: {accuracy_score(y_test,y_pred)}  \n\n")

confusion_matrix(y_test,y_pred)

Accuracy: 0.7619047619047619  




array([[139,  16],
       [ 39,  37]])

In [44]:
from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe, Trials, space_eval
from hyperopt.pyll import scope

def ob(para):
  model = LogisticRegression(**para)
  score = cross_val_score(model,X, y, cv = 5, scoring='accuracy').mean()
  return(-score)

search_spac = {
    'C':hp.loguniform('C',np.log(1e-6),np.log(1e+6)),
    'penalty':hp.choice('penalty',['l2',None]),
    'solver':hp.choice('solver',['lbfgs', 'newton-cg', 'sag', 'saga']),
    'max_iter':scope.int(hp.quniform('max_iter',50,300,1)),
    'multi_class':hp.choice('multi_class',['auto', 'ovr', 'multinomial']),
    'n_jobs':scope.int(hp.quniform('n_jobs',1,6,1)),
    'verbose':hp.choice('verbose',[0,1,2])
}

trials = Trials()

best = fmin(fn = ob, space = search_spac, algo=tpe.suggest, max_evals=100, trials=trials)
best_para = space_eval(search_spac, best)

lr = LogisticRegression(**best_para)
lr.fit(x_train,y_train)
y_pred = lr.predict(x_test)


max_iter reached after 1 seconds
  1%|          | 1/100 [00:00<01:00,  1.64trial/s, best loss: -0.6796706561412443]

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.



max_iter reached after 0 seconds
max_iter reached after 0 seconds
  1%|          | 1/100 [00:01<01:00,  1.64trial/s, best loss: -0.6796706561412443]

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.



max_iter reached after 0 seconds
max_iter reached after 0 seconds
  2%|▏         | 2/100 [00:01<01:10,  1.38trial/s, best loss: -0.6887870299635006]

...

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.



 99%|█████████▉| 99/100 [01:10<00:00,  1.11trial/s, best loss: -0.7734742381801205]

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.



100%|██████████| 100/100 [01:10<00:00,  1.42trial/s, best loss: -0.7734742381801205]


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


In [45]:
accuracy_score(y_test,y_pred)

0.7575757575757576